## Loading Libraries

In [1]:
import pandas as pd
import numpy as np
import torch
import evaluate

c:\Users\Pawel\anaconda3\envs\env_torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading Dataset

In [2]:
df = pd.read_csv('../data/processed/train_sample_processed.csv')

In [3]:
statuses = np.unique(df['OpenStatus'].values)

id2label = {idx: label for idx, label in enumerate(statuses)}
label2id = {label: idx for idx, label in enumerate(statuses)}

In [4]:

from custom_dataset import GithubDataset
from torch.utils.data import random_split
from datasets import Dataset

# Create the pytorch dataset
full_dataset = GithubDataset(df)

In [5]:

# Performing train-test split
TRAIN_DATA_SIZE = int(len(full_dataset) * 0.75)
TEST_DATA_SIZE = int(len(full_dataset) * 0.25)
train_dataset, test_dataset = random_split(full_dataset, [TRAIN_DATA_SIZE, TEST_DATA_SIZE])

train_dataset = Dataset.from_dict(train_dataset[:])
test_dataset = Dataset.from_dict(test_dataset[:])

In [6]:
train_dataset = train_dataset.rename_columns({"text_content": "text", "status": "label"})
test_dataset = test_dataset.rename_columns({"text_content": "text", "status": "label"})

columns_to_remove = [
  'tags_onehot',
  'unrecognized_tags_count',
  'reputation',
  'undeleted_answers',
  'user_life_days',
  'title'
]

train_dataset = train_dataset.remove_columns(columns_to_remove)
test_dataset = test_dataset.remove_columns(columns_to_remove)

## Experimenting With Different Model Architectures

#### COMMENT - IDEAS

We probably should:
- retrain the whole model (probably smaller) with
- better tokenizer - built up from the ground including all the names of the specific tech (languages, frameworks, IDEs, etc.)

### Setting Tokenizer

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [8]:
MAX_TEXT_CONTENT = 128

def tokenize_func(batch):
  tokenized_batch = tokenizer(batch['text'], padding=True, truncation=True, max_length=MAX_TEXT_CONTENT)
  tokenized_batch["label"] = [label2id[label] for label in batch["label"]]
  return tokenized_batch

In [9]:
tokenized_train_dataset = train_dataset.map(tokenize_func, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_func, batched=True)

Map: 100%|██████████| 35068/35068 [00:04<00:00, 7530.17 examples/s]


In [10]:
tokenized_train_dataset.column_names

['text', 'label', 'input_ids', 'attention_mask']

### Preparing Metrics

In [11]:
accuracy_metric = evaluate.load("accuracy")

In [12]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy_metric.compute(predictions=predictions, references=labels)

### Creating Models

In [13]:
from transformers import AutoModelForSequenceClassification

model_content = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(statuses), id2label=id2label, label2id=label2id)
# model_content.to('cuda')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Creating Trainer

In [14]:
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
  output_dir="text_content_model",
  learning_rate=2e-5,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  num_train_epochs=5,
  weight_decay=0.01,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  load_best_model_at_end=True,
)

trainer = Trainer(
  model=model_content,
  args=training_args,
  train_dataset=tokenized_train_dataset,
  eval_dataset=tokenized_test_dataset,
  tokenizer=tokenizer,
  data_collator=data_collator,
  compute_metrics=compute_metrics,
)

In [18]:
from transformers import TrainerCallback
from copy import deepcopy

class CustomCallback(TrainerCallback):
  
  def __init__(self, trainer) -> None:
    super().__init__()
    self._trainer = trainer
  
  def on_epoch_end(self, args, state, control, **kwargs):
    if control.should_evaluate:
      control_copy = deepcopy(control)
      self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
      return control_copy

trainer.add_callback(CustomCallback(trainer)) 

In [19]:
trainer.train('./text_content_model/checkpoint-6576')

 21%|██▏       | 7000/32880 [00:52<10:17, 41.88it/s]   

{'loss': 0.8531, 'learning_rate': 1.5742092457420927e-05, 'epoch': 1.06}


 23%|██▎       | 7500/32880 [01:55<37:11, 11.38it/s]

{'loss': 0.8642, 'learning_rate': 1.5437956204379563e-05, 'epoch': 1.14}


 24%|██▍       | 8000/32880 [02:57<50:45,  8.17it/s]

...

{'loss': 0.8442, 'learning_rate': 1.2396593673965937e-05, 'epoch': 1.9}


 40%|███▉      | 13001/32880 [14:00<42:26,  7.81it/s]

{'loss': 0.8565, 'learning_rate': 1.2092457420924575e-05, 'epoch': 1.98}


                                                     
 40%|████      | 13152/32880 [18:37<41:55,  7.84it/s]

{'train_loss': 0.7580721378326416, 'train_accuracy': 0.7182046310026234, 'train_runtime': 257.2161, 'train_samples_per_second': 409.01, 'train_steps_per_second': 25.566, 'epoch': 2.0}
